## 1.Edit distance with matrix
> 参考视频：https://www.youtube.com/watch?v=We3YDTzNXEk

In [31]:
import numpy as np

In [47]:
class EditDistance:
    def __init__(self):
        self.solution = {}
    @staticmethod
    def edit_distance_with_matrix(str1, str2):
        len_1, len_2 = len(str1) + 1, len(str2) + 1
        if str1 and str2:
            matrix = np.zeros((len_2, len_1), dtype=np.int)
            matrix[0, :] = np.arange(len_1)
            matrix[:, 0] = np.arange(len_2)
            for i, char1 in enumerate(str2):
                for j, char2 in enumerate(str1):
                    if char1 == char2:
                        matrix[i + 1, j + 1] = matrix[i, j]
                    else:
                        matrix[i + 1, j + 1] = min(matrix[i, j + 1], matrix[i, j], matrix[i + 1, j]) + 1

            return matrix[len_2 -1 , len_1 - 1], matrix
        else:
            return len_1 -1  if str1 else len_2 - 1
    
    def get_matrix_solution(self, matrix):
        """
        Up refers to insert; left refers to delete; digonal refers to substating.
        """
        r, c = matrix.shape
        if r < 2:
            for _ in matrix[0, :c-1]:
                self.solution[_+1] = f"delete str1.({_})"
            return
        elif c < 2:
            for _ in matrix[:r-1, 0]:
                self.solution[_ + 1] = f"str1.({0}) + str2.({_})"
            return

        row, column = np.array(matrix.shape) - 1
        target = matrix[row, column]
        up = row - 1, column
        left = row, column - 1
        diag = row - 1, column - 1

        temp = min(matrix[diag], matrix[up], matrix[left])

        # 以下if控制语句的顺序不可变，即从up -> left -> diag顺序判断
        if target == temp + 1:
            if target == matrix[up] + 1:
                self.solution[target] = f"str1.({column - 1}) + str2.({row - 1})"
                return self.get_matrix_solution(matrix[:row, :])
            elif target == matrix[left] + 1:
                self.solution[target] = f"delete str1.({column - 1})"
                return self.get_matrix_solution(matrix[:, :column])
            elif target == matrix[diag] + 1:
                self.solution[target] = f"str1.({column - 1}) -> str2.({row - 1})"
                return self.get_matrix_solution(matrix[:row,:column])
        return self.get_matrix_solution(matrix[:row,:column])
    
    def get_solution(self):
        return sorted(self.solution.items())

In [39]:
test = EditDistance()
step, matrix_ = test.edit_distance_with_matrix('welecome', 'stinfwelcom')
test.get_matrix_solution(matrix_)
test.get_solution()

[(1, 'str1.(0) + str2.(0)'),
 (2, 'str1.(0) + str2.(1)'),
 (3, 'str1.(0) + str2.(2)'),
 (4, 'str1.(0) + str2.(3)'),
 (5, 'str1.(0) + str2.(4)'),
 (6, 'delete str1.(3)'),
 (7, 'delete str1.(7)')]

In [58]:
test_ = EditDistance()
step, matrix_ = test_.edit_distance_with_matrix('aebd', 'abcd')
test_.get_matrix_solution(matrix_)
print(matrix_)
print(step)
test_.get_solution()

[[0 1 2 3 4]
 [1 0 1 2 3]
 [2 1 1 1 2]
 [3 2 2 2 2]
 [4 3 3 3 2]]
2


[(1, 'delete str1.(1)'), (2, 'str1.(2) + str2.(2)')]

## 2.Edit distance with recursion

In [2]:
from functools import lru_cache

In [79]:
if 'solution_' in dir(): del solution_ 
solution_ = {}
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    if string1 and string2:
    
        tail_s1 = string1[-1]
        tail_s2 = string2[-1]

        candidates = [
            (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  # string 1 delete tail
            (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  # string 1 add tail of string2
        ]

        if tail_s1 == tail_s2:
            both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
        else:
            both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

        candidates.append(both_forward)

        min_distance, operation = min(candidates, key=lambda x: x[0])

        solution_[(string1, string2)] = operation 
        print(candidates)
        return min_distance
    
    return len(string1) if string1 else len(string2)

In [80]:
edit_distance('aebd', 'abcd')

[(2, 'DEL a'), (2, 'ADD a'), (0, '')]
[(3, 'DEL a'), (1, 'ADD b'), (2, 'SUB a => b')]
[(4, 'DEL a'), (2, 'ADD c'), (3, 'SUB a => c')]
[(5, 'DEL a'), (3, 'ADD d'), (4, 'SUB a => d')]
[(1, 'DEL e'), (3, 'ADD a'), (2, 'SUB e => a')]
[(2, 'DEL e'), (2, 'ADD b'), (1, 'SUB e => b')]
[(3, 'DEL e'), (2, 'ADD c'), (2, 'SUB e => c')]
[(4, 'DEL e'), (3, 'ADD d'), (3, 'SUB e => d')]
[(2, 'DEL b'), (4, 'ADD a'), (3, 'SUB b => a')]
[(2, 'DEL b'), (3, 'ADD b'), (1, '')]
[(3, 'DEL b'), (2, 'ADD c'), (2, 'SUB b => c')]
[(4, 'DEL b'), (3, 'ADD d'), (3, 'SUB b => d')]
[(3, 'DEL d'), (5, 'ADD a'), (4, 'SUB d => a')]
[(2, 'DEL d'), (4, 'ADD b'), (3, 'SUB d => b')]
[(3, 'DEL d'), (3, 'ADD c'), (2, 'SUB d => c')]
[(4, 'DEL d'), (3, 'ADD d'), (2, '')]


2

In [75]:
solution_

{('a', 'a'): '',
 ('a', 'ab'): 'ADD b',
 ('a', 'abc'): 'ADD c',
 ('a', 'abcd'): 'ADD d',
 ('ae', 'a'): 'DEL e',
 ('ae', 'ab'): 'SUB e => b',
 ('ae', 'abc'): 'ADD c',
 ('ae', 'abcd'): 'ADD d',
 ('aeb', 'a'): 'DEL b',
 ('aeb', 'ab'): '',
 ('aeb', 'abc'): 'ADD c',
 ('aeb', 'abcd'): 'ADD d',
 ('aebd', 'a'): 'DEL d',
 ('aebd', 'ab'): 'DEL d',
 ('aebd', 'abc'): 'SUB d => c',
 ('aebd', 'abcd'): ''}

In [76]:
edit_distance('c1297BCF9', '8899cABCDEFHNEDF')

12

In [57]:
solution_

{('c', '8'): 'SUB c => 8',
 ('c', '88'): 'ADD 8',
 ('c', '889'): 'ADD 9',
 ('c', '8899'): 'ADD 9',
 ('c', '8899c'): '',
 ('c', '8899cA'): 'ADD A',
 ('c', '8899cAB'): 'ADD B',
 ('c', '8899cABC'): 'ADD C',
 ('c', '8899cABCD'): 'ADD D',
 ('c', '8899cABCDE'): 'ADD E',
 ('c', '8899cABCDEF'): 'ADD F',
 ('c', '8899cABCDEFH'): 'ADD H',
 ('c', '8899cABCDEFHN'): 'ADD N',
 ('c', '8899cABCDEFHNE'): 'ADD E',
 ('c', '8899cABCDEFHNED'): 'ADD D',
 ('c', '8899cABCDEFHNEDF'): 'ADD F',
 ('c1', '8'): 'DEL 1',
 ('c1', '88'): 'SUB 1 => 8',
 ('c1', '889'): 'ADD 9',
 ('c1', '8899'): 'ADD 9',
 ('c1', '8899c'): 'DEL 1',
 ('c1', '8899cA'): 'SUB 1 => A',
 ('c1', '8899cAB'): 'ADD B',
 ('c1', '8899cABC'): 'ADD C',
 ('c1', '8899cABCD'): 'ADD D',
 ('c1', '8899cABCDE'): 'ADD E',
 ('c1', '8899cABCDEF'): 'ADD F',
 ('c1', '8899cABCDEFH'): 'ADD H',
 ('c1', '8899cABCDEFHN'): 'ADD N',
 ('c1', '8899cABCDEFHNE'): 'ADD E',
 ('c1', '8899cABCDEFHNED'): 'ADD D',
 ('c1', '8899cABCDEFHNEDF'): 'ADD F',
 ('c12', '8'): 'DEL 2',
 ('c12

## 3.Longest common substring with matrix
> 最长公共子串

In [56]:
def longest_substring_matrix(str1, str2):
    """
    Return all the longest common substring of two strings.
    """
    result = []
    row, column = len(str1), len(str2)
    matrix = np.zeros((row, column), dtype=int)
    for i, char1 in enumerate(str1):
        for j, char2 in enumerate(str2):
            if char1 == char2:
                matrix[i, j] = matrix[i-1, j-1] + 1
    print(matrix)
    print(args)
    for arg in args:
        index_ = arg[0]
        result.append(str1[index_ - max_ + 1 : index_ + 1])
    return max_, set(result)

In [57]:
longest_substring_matrix('hellowel', 'hellfadellolflowedlfalowefsello')

[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 2 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0]
 [0 0 3 1 0 0 0 0 2 1 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 2 1 0]
 [0 0 1 4 0 0 0 0 1 3 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 3 0]
 [0 0 0 0 0 0 0 0 0 0 4 0 0 0 2 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 4]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 4 0 0 1 0 0 0]
 [0 0 2 1 0 0 0 0 2 1 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 2 1 0]]
[[ 3  3]
 [ 4 10]
 [ 4 30]
 [ 6 16]
 [ 6 24]]


(4, {'ello', 'hell', 'lowe'})

In [58]:
longest_substring_matrix('fish', 'fhish')

[[1 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 2 0]
 [0 1 0 0 3]]
[[3 4]]


(3, {'ish'})

## 4.Longest common subsequence with matrix
> 最长公共子序列

In [59]:
def longest_subsequence_matrix(str1, str2):
    """
    Return the longest common subsequence of two strings, return one of them if there are mutiple solution with same 
    length.
    """
    str1, str2 = ' '+ str1, ' '+ str2
    index_ = []
    len_1, len_2 = len(str1), len(str2)
    if str1 and str2:
        matrix = np.zeros((len_1, len_2), dtype=np.int)
        for i, char1 in enumerate(str1):
            if i == 0: continue
            for j, char2 in enumerate(str2):
                if j == 0: continue
                if char1 == char2:
                    matrix[i, j] = matrix[i - 1, j - 1] + 1
                else:
                    matrix[i, j] = max(matrix[i, j - 1], matrix[i - 1, j])
                    
        target = np.max(matrix)
        
        for i in range(1, target + 1):
            # 找出第一个加1的数
            # 行小于列，以行索引取最小，反之，以列索引取最小
            flag = 0 if len_1 < len_2 else 1
            temp = min(np.argwhere(matrix == i), key = lambda x: x[flag])
            index_.append(temp)  
            
        result = [str1[i[0]] for i in index_]
        print(matrix)
        return target, result
    else:
        return len_1 -1  if str1 else len_2 - 1

In [60]:
longest_subsequence_matrix('h1e2ll3o2y34o234u', 'helloyou')

[[0 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1 1]
 [0 1 2 2 2 2 2 2 2]
 [0 1 2 2 2 2 2 2 2]
 [0 1 2 3 3 3 3 3 3]
 [0 1 2 3 4 4 4 4 4]
 [0 1 2 3 4 4 4 4 4]
 [0 1 2 3 4 5 5 5 5]
 [0 1 2 3 4 5 5 5 5]
 [0 1 2 3 4 5 6 6 6]
 [0 1 2 3 4 5 6 6 6]
 [0 1 2 3 4 5 6 6 6]
 [0 1 2 3 4 5 6 7 7]
 [0 1 2 3 4 5 6 7 7]
 [0 1 2 3 4 5 6 7 7]
 [0 1 2 3 4 5 6 7 7]
 [0 1 2 3 4 5 6 7 8]]


(8, ['h', 'e', 'l', 'l', 'o', 'y', 'o', 'u'])

In [61]:
longest_subsequence_matrix('ABCDEFG', 'BCDAFG')

[[0 0 0 0 0 0 0]
 [0 0 0 0 1 1 1]
 [0 1 1 1 1 1 1]
 [0 1 2 2 2 2 2]
 [0 1 2 3 3 3 3]
 [0 1 2 3 3 3 3]
 [0 1 2 3 3 4 4]
 [0 1 2 3 3 4 5]]


(5, ['B', 'C', 'D', 'F', 'G'])

## 5.dijsktra算法
![](./graph.png)

In [15]:
inf = float('inf')

In [16]:
graph = {
    'start': {'a': 6, 'b': 2},
    'a': {'fin' : 1},
    'b': {'a': 3, 'fin': 5},
    'fin': {}
}
costs = {
    'a': 6,
    'b': 2,
    'fin': inf
}
parents = {
    'a': 'start',
    'b': 'start',
    'fin': None
}
processed = []

In [19]:
# del r
r = ['start']
def find_lowest_cost_node(costs):
    result = list(filter(lambda x: x[0] not in processed, sorted(costs.items(), key=lambda x: x[1])))
    return result[0][0] if result else None

def get_next_(result, key='start'):
    for k, v in result.items():
        if k == key:
            r.append(v)
            get_next_(result, v)
    return r
        
def dijsktra_algorithm(graph=graph, costs=costs, parents=parents):
    node = find_lowest_cost_node(costs)
    result = {}
    while node is not None:
        cost = costs[node]
        neighbors = graph[node]
        for n in neighbors.keys():
            new_cost = cost + neighbors[n]
            if costs[n] > new_cost:
                costs[n] = new_cost
                parents[n] = node
        processed.append(node)
        node = find_lowest_cost_node(costs)
    result = {v:k for k,v in parents.items()}
    result = get_next_(result)
    return ' -> '.join(result), costs['fin']

In [20]:
dijsktra_algorithm()

('start -> b -> a -> fin', 6)

In [42]:
np.random.seed(1)
y = np.random.randn(10)
np.random.seed(2)
z = np.random.randn(10)
np.random.seed(1)
x = np.random.randn(10)

In [43]:
x

array([ 1.62434536, -0.61175641, -0.52817175, -1.07296862,  0.86540763,
       -2.3015387 ,  1.74481176, -0.7612069 ,  0.3190391 , -0.24937038])

In [44]:
y

array([ 1.62434536, -0.61175641, -0.52817175, -1.07296862,  0.86540763,
       -2.3015387 ,  1.74481176, -0.7612069 ,  0.3190391 , -0.24937038])

In [45]:
z

array([-0.41675785, -0.05626683, -2.1361961 ,  1.64027081, -1.79343559,
       -0.84174737,  0.50288142, -1.24528809, -1.05795222, -0.90900761])

In [46]:
np.random.seed(2)
z1= np.random.randn(10)

In [47]:
z1

array([-0.41675785, -0.05626683, -2.1361961 ,  1.64027081, -1.79343559,
       -0.84174737,  0.50288142, -1.24528809, -1.05795222, -0.90900761])